## Estrutura de uma DAG no Airflow

Uma **DAG (Directed Acyclic Graph)** no Airflow define um pipeline de tarefas. É composta por um conjunto de tarefas e as dependências entre elas. A DAG determina a ordem e as condições em que as tarefas devem ser executadas.

As principais estruturas de uma DAG incluem:

- **`dag_id`**: Um identificador único para a DAG.
- **`schedule_interval`**: Define a frequência com que a DAG deve ser executada (diariamente, semanalmente, etc.).
- **`default_args`**: Argumentos padrões, como o horário de início, dependências, e se a DAG deve ser retroativa.
- **`tasks`**: Tarefas que são executadas dentro da DAG.
- **Dependências entre tarefas**: Definem a ordem de execução entre as tarefas.

---

**Exemplo básico de uma DAG:**

1. Definir a DAG e seus argumentos básicos.
2. Criar tarefas individuais.
3. Definir dependências entre as tarefas.


In [ ]:
# Exemplo de DAG simples no Airflow

from airflow import DAG
from airflow.operators.dummy import DummyOperator
from datetime import datetime, timedelta

# Definindo os argumentos padrão da DAG
default_args = {
    'owner': 'maicon',
    'depends_on_past': False,
    'start_date': datetime(2023, 9, 1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Definindo a DAG
with DAG(
    dag_id='exemplo_dag',
    default_args=default_args,
    description='Exemplo de uma DAG simples',
    schedule_interval=timedelta(days=1),
    catchup=False,  # Não executa execuções passadas
) as dag:

    # Criando as tarefas
    tarefa_inicial = DummyOperator(task_id='inicio')
    tarefa_final = DummyOperator(task_id='fim')

    # Definindo as dependências
    tarefa_inicial >> tarefa_final  # 'inicio' deve rodar antes de 'fim'


## Principais Operadores no Airflow

No Airflow, os **operadores** são os blocos de construção usados para definir tarefas dentro de um DAG (Directed Acyclic Graph). Cada operador representa uma única tarefa e define o que deve ser executado. Os operadores podem ser divididos em várias categorias, dependendo de suas funcionalidades, como executar comandos Bash, funções Python, transferir dados ou executar consultas SQL.

### Principais operadores no Airflow:
1. **BashOperator**: Executa comandos ou scripts bash diretamente na linha de comando.
2. **PythonOperator**: Executa funções Python. É útil para executar lógica Python personalizada dentro de um DAG.
3. **EmailOperator**: Envia emails. Geralmente usado para notificar falhas ou sucessos de tarefas.
4. **PostgresOperator**: Executa comandos SQL em um banco de dados Postgres.
5. **MySqlOperator**: Executa comandos SQL em um banco de dados MySQL.
6. **S3ToGCSOperator**: Transfere arquivos do AWS S3 para o Google Cloud Storage (GCS).
7. **SimpleHttpOperator**: Faz solicitações HTTP. Útil para chamadas de API.
8. **DummyOperator**: Representa uma tarefa "sem operação", geralmente usada para testes ou como marcadores em um DAG.


In [ ]:
# Exemplo de DAG usando diferentes operadores no Airflow
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
from airflow.operators.dummy_operator import DummyOperator
from airflow.utils.dates import days_ago

# Função Python a ser usada no PythonOperator
def print_hello():
    print("Hello from Python!")

# Definindo o DAG
default_args = {
    'owner': 'airflow',
    'start_date': days_ago(1),
}
dag = DAG(
    'exemplo_dag',
    default_args=default_args,
    schedule_interval='@daily',
)

# Criando tarefas com diferentes operadores

# BashOperator - Executa um comando Bash
bash_task = BashOperator(
    task_id='executar_script_bash',
    bash_command='echo "Hello Airflow!"',
    dag=dag
)

# PythonOperator - Executa uma função Python
python_task = PythonOperator(
    task_id='executar_funcao_python',
    python_callable=print_hello,
    dag=dag
)

# DummyOperator - Não faz nada, usado como placeholder
dummy_task = DummyOperator(
    task_id='tarefa_placeholder',
    dag=dag
)

# Definindo dependências entre as tarefas
bash_task >> python_task >> dummy_task
